<a href="https://colab.research.google.com/github/GrigoreGeorgeAlexandru/Colab-projects/blob/main/Proiect_BIGDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cuprins:
1. Introducere
2. Implementarea de scripturi Spark
3. Aplicarea metode ML
4. Data Pipeline
5. functii UDF
6. Aplicarea metode DL
7. Spark Streaming

1.	Introducere

Pentru proiect am ales setul de date “IMDB Top 250 TV Shows” , ce ofera o lista de seriale TV si informatii despre acestea asa cum sunt prezentate pe site-ul IMDB: titlu, an, numarul de episoade , varsta recomandata, rating-ul,numarul de voturi primate pe site, categoria de serial.

Link: https://www.kaggle.com/datasets/ritiksharma07/top-250-imdb-tv-shows?resource=download

Obiectivele proiectului sunt rezolvarea cerintelor de proiect: implementarea de scripturi Spark,aplicarea a doua metode ML, folosirea unui Data Pipeline,utilizarea unei functii UDF, aplicarea unei metode DL,creerea unui process de streaming folosind Spark Streaming.


2.Implementarea de scripturi Spark


In [ ]:
pip install pyspark


In [ ]:
import pyspark
from pyspark import SparkContext as sc
import numpy as np
from pyspark.sql import SparkSession

In [ ]:

spark1 = SparkSession.builder.appName('Project').getOrCreate()
df = spark1.read.csv('IMDB_Top250_Tvshows.csv') #citim setul de date


In [ ]:
df.show() #vedem starea initiala a datelor


+--------------------+---------+--------------+---+------+----------+--------------+
|                 _c0|      _c1|           _c2|_c3|   _c4|       _c5|           _c6|
+--------------------+---------+--------------+---+------+----------+--------------+
|              Titile|     Year|Total_episodes|Age|Rating|Vote_count|      Category|
|     1. Breaking Bad|2008�2013|        62 eps| 18|   9.5|    (2.2M)|     TV Series|
|  2. Planet Earth II|     2016|         6 eps| PG|   9.5|    (159K)|TV Mini Series|
|     3. Planet Earth|     2006|        11 eps| PG|   9.4|    (221K)|TV Mini Series|
| 4. Band of Brothers|     2001|        10 eps| 15|   9.4|    (533K)|TV Mini Series|
|        5. Chernobyl|     2019|         5 eps| 15|   9.3|    (876K)|TV Mini Series|
|         6. The Wire|2002�2008|        60 eps| 18|   9.3|    (381K)|     TV Series|
|7. Avatar: The La...|2005�2008|        62 eps| PG|   9.3|    (378K)|     TV Series|
|   8. Blue Planet II|     2017|         7 eps|  U|   9.3|     (4

In [ ]:
#definim propria schema pentru setul de date
from pyspark.sql.types import StructField, IntegerType, StringType, StructType, FloatType
data_schema = [StructField('Title',StringType(),True),
StructField('Year',StringType(),True),StructField('Total episodes',StringType(),True),StructField('Age',StringType(),True),
               StructField('Rating',FloatType(),True),StructField('Vote count',StringType(),True),StructField('Category',StringType(),True),]
final_struct = StructType(fields=data_schema)

In [ ]:
#creeam dataframe-ul
df = spark1.read.csv('IMDB_Top250_Tvshows.csv',schema=final_struct)
df.show()
#curatam datele de primul rand care nu ne mai foloseste
df = df.dropna()
df.show()

#transformam coloana de episoade din string in int, eliminand partea cu "eps"
from pyspark.sql.functions import regexp_extract, col
df = df.withColumn("Total episodes", regexp_extract(col("Total episodes"), r"(\d+)", 1).cast("int"))
df.show()



+--------------------+---------+--------------+---+------+----------+--------------+
|               Title|     Year|Total episodes|Age|Rating|Vote count|      Category|
+--------------------+---------+--------------+---+------+----------+--------------+
|              Titile|     Year|Total_episodes|Age|  NULL|Vote_count|      Category|
|     1. Breaking Bad|2008�2013|        62 eps| 18|   9.5|    (2.2M)|     TV Series|
|  2. Planet Earth II|     2016|         6 eps| PG|   9.5|    (159K)|TV Mini Series|
|     3. Planet Earth|     2006|        11 eps| PG|   9.4|    (221K)|TV Mini Series|
| 4. Band of Brothers|     2001|        10 eps| 15|   9.4|    (533K)|TV Mini Series|
|        5. Chernobyl|     2019|         5 eps| 15|   9.3|    (876K)|TV Mini Series|
|         6. The Wire|2002�2008|        60 eps| 18|   9.3|    (381K)|     TV Series|
|7. Avatar: The La...|2005�2008|        62 eps| PG|   9.3|    (378K)|     TV Series|
|   8. Blue Planet II|     2017|         7 eps|  U|   9.3|     (4

In [ ]:


#grupari si agregari de date

df.filter((df['Total episodes']<30)).select(['Title','Rating','Age']).show(5)# afisam primele 10 intrari pentru seriale cu sub 30 episoade
df.filter((df['Total episodes']<30) & (df['Rating']>7)).show(5)#afisam primele 5 intrari care au sub 30 ep si nota peste 7
df.filter(df['Category']=="TV Mini Series").show() #afisam toate serialele mini series

#luam toate serialele cu rating-ul PG si afisam primul rand ca dictionar
data1 = df.filter(df['Age'] == "PG").collect()
row=data1[0].asDict()
print(row)
#luam toate serialele cu nota peste 9 si afisam primul rand ca dictionar
data2 = df.filter(df['Age'] > 9).collect()
row=data2[0].asDict()
print(row)

+-------------------+------+---+
|              Title|Rating|Age|
+-------------------+------+---+
| 2. Planet Earth II|   9.5| PG|
|    3. Planet Earth|   9.4| PG|
|4. Band of Brothers|   9.4| 15|
|       5. Chernobyl|   9.3| 15|
|  8. Blue Planet II|   9.3|  U|
+-------------------+------+---+
only showing top 5 rows

+-------------------+----+--------------+---+------+----------+--------------+
|              Title|Year|Total episodes|Age|Rating|Vote count|      Category|
+-------------------+----+--------------+---+------+----------+--------------+
| 2. Planet Earth II|2016|             6| PG|   9.5|    (159K)|TV Mini Series|
|    3. Planet Earth|2006|            11| PG|   9.4|    (221K)|TV Mini Series|
|4. Band of Brothers|2001|            10| 15|   9.4|    (533K)|TV Mini Series|
|       5. Chernobyl|2019|             5| 15|   9.3|    (876K)|TV Mini Series|
|  8. Blue Planet II|2017|             7|  U|   9.3|     (47K)|TV Mini Series|
+-------------------+----+--------------+---+-

In [ ]:
#SparkSQL


df.createOrReplaceTempView("data")
#afisam primele 5 randuri din data
result = spark1.sql("SELECT * FROM data LIMIT 5")
result

#Afisam coloanele rezultatului
result.columns
#Afisam rezultatul
result.show()
#afisam primele 3 coloane din setul de date denumite Title Year Episode Count
data = spark1.sql("SELECT _c0 as Title, _c1 as Year, _c2 as Episode_Count FROM csv.`IMDB_Top250_Tvshows.csv` offset 1")
data.show()
#afisam primele 10 seriale cu rating-ul peste 9
spark1.sql("SELECT * FROM data WHERE rating>9 LIMIT 10").show()




+-------------------+---------+--------------+---+------+----------+--------------+
|              Title|     Year|Total episodes|Age|Rating|Vote count|      Category|
+-------------------+---------+--------------+---+------+----------+--------------+
|    1. Breaking Bad|2008�2013|            62| 18|   9.5|    (2.2M)|     TV Series|
| 2. Planet Earth II|     2016|             6| PG|   9.5|    (159K)|TV Mini Series|
|    3. Planet Earth|     2006|            11| PG|   9.4|    (221K)|TV Mini Series|
|4. Band of Brothers|     2001|            10| 15|   9.4|    (533K)|TV Mini Series|
|       5. Chernobyl|     2019|             5| 15|   9.3|    (876K)|TV Mini Series|
+-------------------+---------+--------------+---+------+----------+--------------+

+--------------------+---------+-------------+
|               Title|     Year|Episode_Count|
+--------------------+---------+-------------+
|     1. Breaking Bad|2008�2013|       62 eps|
|  2. Planet Earth II|     2016|        6 eps|
|     3.

In [ ]:
#facem grupari de date si afisam rezultatele

df.groupby('Category').mean().show()#medie
df.groupby('Category').max().show()#maxim
df.groupby('Category').min().show()#minim


#aceleasi operatii cu agregari
df_group=df.groupby('Category')
df_group.agg({'Rating':'max'}).show()#medie
df_group.agg({'Rating':'min'}).show()#maxim
df_group.agg({'Rating':'mean'}).show()#minim

#afisam categoria de serial cu cea mai mare valoare de medie de rating-uri
spark1.sql('''
select Category, mean(Rating) as mean_rating from data
group by Category
having mean_rating = (select max(me) from (
  select mean(Rating) as me from data group by Category
    ) )
''').show()

+--------------+-------------------+-----------------+
|      Category|avg(Total episodes)|      avg(Rating)|
+--------------+-------------------+-----------------+
|     TV Series|   84.3170731707317|8.731219561507062|
|TV Mini Series|  8.179487179487179|8.887179545867138|
+--------------+-------------------+-----------------+

+--------------+-------------------+-----------+
|      Category|max(Total episodes)|max(Rating)|
+--------------+-------------------+-----------+
|     TV Series|               1122|        9.5|
|TV Mini Series|                 13|        9.5|
+--------------+-------------------+-----------+

+--------------+-------------------+-----------+
|      Category|min(Total episodes)|min(Rating)|
+--------------+-------------------+-----------+
|     TV Series|                  4|        8.5|
|TV Mini Series|                  2|        8.5|
+--------------+-------------------+-----------+

+--------------+-----------+
|      Category|max(Rating)|
+--------------+-----

In [ ]:

df.show()
# Importați clasa LinearRegression
from pyspark.ml.regression import LinearRegression
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#regresia liniara pentru a prezice rating-ul in functie de numarul de episoade:
# Adaugam coloana total episodes in vectorul de caracteristici
assembler = VectorAssembler(
    inputCols=["Total episodes"
               ],
    outputCol="features")
output = assembler.transform(df)
#Afișați coloana features
output.select("features").show(truncate=False)
output.select("features").head(3)
output.show()
#Afișați toate coloanele din output
output.columns
# Reținem doar coloana Rating
final_data = output.select("features",'Rating')
# Creați o diviziune train-test cu procentele 70%-30%
train_data,test_data = output.randomSplit([0.7,0.3])
# Afișați statistici asupra datelor de training
train_data.summary().show()
# Afișați statistici asupra datelor de test
test_data.summary().show()
# Creați un model de regresie liniară
lrModel=LinearRegression(labelCol="Rating")
# Antrenați modelul
lrModel = lrModel.fit(train_data)
# Afișați coeficienții și intercepția pentru regresia liniară
lrModel.coefficients,lrModel.intercept
#Evaluați modelul pe datele de test
test_results=lrModel.evaluate(test_data)

# Afișați valorile reziduale
test_results.residuals.show()
# Obțineți datele de test neetichetate.
unlabled_data=test_data.select('features')
# Aplicați modelul obținut anterior pe datele de test neetichetate
predictions=lrModel.transform(unlabled_data)
#Afișați predicțiile obținute
predictions.show()
#comparam cu datele test
test_data.select("features","Rating").show()


+--------------------+---------+--------------+---+------+----------+--------------+
|               Title|     Year|Total episodes|Age|Rating|Vote count|      Category|
+--------------------+---------+--------------+---+------+----------+--------------+
|     1. Breaking Bad|2008�2013|            62| 18|   9.5|    (2.2M)|     TV Series|
|  2. Planet Earth II|     2016|             6| PG|   9.5|    (159K)|TV Mini Series|
|     3. Planet Earth|     2006|            11| PG|   9.4|    (221K)|TV Mini Series|
| 4. Band of Brothers|     2001|            10| 15|   9.4|    (533K)|TV Mini Series|
|        5. Chernobyl|     2019|             5| 15|   9.3|    (876K)|TV Mini Series|
|         6. The Wire|2002�2008|            60| 18|   9.3|    (381K)|     TV Series|
|7. Avatar: The La...|2005�2008|            62| PG|   9.3|    (378K)|     TV Series|
|   8. Blue Planet II|     2017|             7|  U|   9.3|     (47K)|TV Mini Series|
|     9. The Sopranos|1999�2007|            86| 18|   9.2|    (47

In [ ]:
#regresia logistica


from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType





# Definim o funcție UDF ca sa transformam coloana Age in int

def is_rating(rating):


  if rating == 'E':
    return 0
  elif rating == 'U':
    return 0
  elif rating == 'G':
    return 0
  elif rating == 'PG':
    return 13
  elif rating == '12A':
    return 12
  elif rating == '12':
    return 12
  elif rating == '15':
    return 15
  elif rating == '18':
    return 18
  elif rating == 'Caution':
    return 18
  elif rating == 'R18':
    return 18



# Înregistrăm funcția ca UDF și specificăm tipul de returnare
is_rating_udf = udf(is_rating, IntegerType())

# Aplicăm UDF-ul pe coloana "age" și creăm o nouă coloană
df = df.withColumn("Age", is_rating_udf(df["Age"]))
df.show()
df.printSchema()



#transformam coloana de rating in int-uri pentru regresia logistica
df = df.withColumn('Rating', col('Rating').cast("int"))
df.show()
df.printSchema()

#creama age indexer si episode indexer
age_indexer = StringIndexer(inputCol='Age',outputCol='AgeIndex', handleInvalid="keep")
age_encoder = OneHotEncoder(inputCol='AgeIndex',outputCol='AgeVec', handleInvalid="keep")

episode_indexer = StringIndexer(inputCol='Total episodes',outputCol='EpisodeIndex', handleInvalid="keep")
episode_encoder = OneHotEncoder(inputCol='EpisodeIndex',outputCol='EpisodeVec', handleInvalid="keep")
#Asamblati coloanele setului de date in vectorul "features"
assembler = VectorAssembler(inputCols=[ 'EpisodeVec', 'AgeVec'],outputCol='features')

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

#incepem regresia
log_data = LogisticRegression(featuresCol='features',labelCol='Rating')

#Inlantui elementele definite mai sus ca etape in cadrul unui Pipeline
pipeline =Pipeline(stages=[age_indexer,episode_indexer, age_encoder,episode_encoder,assembler,log_data])

#curatam si antrenam datele
my_final_data = df.select([  'Total episodes',  'Age',  'Rating'])
my_final_data = my_final_data.na.drop()
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

#Antrenare pipeline
fit_model = pipeline.fit(train_data)

#Aplicare model pe datele de test
results = fit_model.transform(test_data)

from pyspark.ml.evaluation import BinaryClassificationEvaluator

#evaluare
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Rating')

results.printSchema()
results.select('Rating','prediction').show()

AUC = my_eval.evaluate(results)

AUC




+--------------------+---------+--------------+---+------+----------+--------------+
|               Title|     Year|Total episodes|Age|Rating|Vote count|      Category|
+--------------------+---------+--------------+---+------+----------+--------------+
|     1. Breaking Bad|2008�2013|            62| 18|   9.5|    (2.2M)|     TV Series|
|  2. Planet Earth II|     2016|             6| 13|   9.5|    (159K)|TV Mini Series|
|     3. Planet Earth|     2006|            11| 13|   9.4|    (221K)|TV Mini Series|
| 4. Band of Brothers|     2001|            10| 15|   9.4|    (533K)|TV Mini Series|
|        5. Chernobyl|     2019|             5| 15|   9.3|    (876K)|TV Mini Series|
|         6. The Wire|2002�2008|            60| 18|   9.3|    (381K)|     TV Series|
|7. Avatar: The La...|2005�2008|            62| 13|   9.3|    (378K)|     TV Series|
|   8. Blue Planet II|     2017|             7|  0|   9.3|     (47K)|TV Mini Series|
|     9. The Sopranos|1999�2007|            86| 18|   9.2|    (47

1.0